<a href="https://colab.research.google.com/github/CrissRMFI/TA047R-2C2024-GRUPO08/blob/main/TA047R_TP2_GRUPO08_ENTREGA_N4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

In [ ]:
train_set = pd.read_csv("./train.csv")
test_set = pd.read_csv("./test.csv")

# ANÁLISIS EXPLORATORIO

In [ ]:
train_set.head(10)

,id,title,description,project,storypoint
0,5660,Error enabling Appcelerator services during ap...,"When creating the default app, I encountered t...",project8,3
1,9014,Create a maintenance branch,"As a developer, I'd like to have a maintenance...",project6,5
2,4094,Service Activity Monitoring Backend integrated...,SAM API used by SAM GUI,project1,5
3,811,fs::enter(rootfs) does not work if 'rootfs' is...,I noticed this when I was testing the unified ...,project5,2
4,4459,transform processor with script option is broken,Creating the following stream throws exception...,project6,2
5,8896,Service Registry: REST endpoints URL renaming,Endpoint URLs of Service Registry REST service...,project1,1
6,2304,TiStudio: Projects - A pane is missing when cr...,Details: If you try to create the default Tita...,project8,5
7,9192,Avoid installing test and performance services...,The membership plans are associated with organ...,project8,5
8,1888,Add Unit Tests for CC SPI infrastructure,"Test Converter, Configuration, Definition and ...",project6,3
9,4609,Agent should not return partial state when a r...,Currently when a user is hitting /state.json o...,project5,3


In [ ]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7900 entries, 0 to 7899
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           7900 non-null   int64 
 1   title        7900 non-null   object
 2   description  7900 non-null   object
 3   project      7900 non-null   object
 4   storypoint   7900 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 308.7+ KB


In [ ]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1975 entries, 0 to 1974
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           1975 non-null   int64 
 1   title        1975 non-null   object
 2   description  1975 non-null   object
 3   project      1975 non-null   object
dtypes: int64(1), object(3)
memory usage: 61.8+ KB


Verificar si posee datos nulos

In [ ]:
print(train_set.isna().sum() / train_set.shape[0] * 100)

id             0.0
title          0.0
description    0.0
project        0.0
storypoint     0.0
dtype: float64


In [ ]:
print(test_set.isna().sum() / test_set.shape[0] * 100)

id             0.0
title          0.0
description    0.0
project        0.0
dtype: float64


# Pre-Procesamiento

# Separamos los datos en variables de entrada y etiquetas

In [ ]:
X_test_text = test_set[['title','description','project']]

In [ ]:
X_train_text = train_set[['title','description','project']]
Y_train = train_set['storypoint']

In [ ]:
X_train_text

,title,description,project
0,Error enabling Appcelerator services during ap...,"When creating the default app, I encountered t...",project8
1,Create a maintenance branch,"As a developer, I'd like to have a maintenance...",project6
2,Service Activity Monitoring Backend integrated...,SAM API used by SAM GUI,project1
3,fs::enter(rootfs) does not work if 'rootfs' is...,I noticed this when I was testing the unified ...,project5
4,transform processor with script option is broken,Creating the following stream throws exception...,project6
...,...,...,...
7895,"As Patrick, I want to be able to create a new ...",# Dialog is shown # fields we need to populat...,project7
7896,GMock warning in ReservationTest.ACLMultipleOp...,{noformat} [ RUN ] ReservationTest.ACLMu...,project5
7897,WSDL Improvement of the SOAP based Service Loc...,As developer I want to use a service that foll...,project1
7898,Platform Config Wizard: Auto-check if Tizen is...,This is an extension of TISTUD-5246. When the ...,project8


## Vectorizamos el texto

In [ ]:
import nltk
from nltk import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
posImportantes = ["ADJ","ADV","NOUN","VERB","INTJ"]
def dejarPalabras(texto):
  textoTokenizado = nlp(texto)
  palabras = [];
  for token in textoTokenizado:
    if(token.is_alpha and not token.is_stop and not token.is_punct):
      palabras.append(token.lemma_.lower());
  frecuencia = Counter(palabras)
  return frecuencia.most_common(5)

In [ ]:
X_train_token = X_train_text.copy()
X_train_token['palabras descripcion'] = X_train_text['description'].map(lambda x : dejarPalabras(x))
X_test_token = X_test_text.copy()
X_test_token['palabras descripcion'] = X_test_text['description'].map(lambda x : dejarPalabras(x))

In [ ]:
X_train_token['palabras titulo'] = X_train_text['title'].map(lambda x : dejarPalabras(x))
X_test_token['palabras titulo'] = X_test_text['title'].map(lambda x : dejarPalabras(x))

In [ ]:
X_train_token.head(3)

,title,description,project,palabras descripcion,palabras titulo
0,Error enabling Appcelerator services during ap...,"When creating the default app, I encountered t...",project8,"[(create, 1), (default, 1), (app, 1), (encount...","[(error, 1), (enable, 1), (appcelerator, 1), (..."
1,Create a maintenance branch,"As a developer, I'd like to have a maintenance...",project6,"[(commit, 2), (developer, 1), (like, 1), (main...","[(create, 1), (maintenance, 1), (branch, 1)]"
2,Service Activity Monitoring Backend integrated...,SAM API used by SAM GUI,project1,"[(sam, 2), (api, 1), (gui, 1)]","[(service, 1), (activity, 1), (monitoring, 1),..."


In [ ]:
indexPalabrasDescripcion = {};
indexPalabrasTitulo = {};
def agregarYDevolverIndex(listaPalabras, indexPalabras, palabraClave):
  index = [];
  for palabra in listaPalabras:
    if (palabra not in indexPalabras):
      indexPalabras[palabra] = str(len(indexPalabras))+palabraClave
    index.append(indexPalabras[palabra]);
  return index;

In [ ]:
palabraClaveTitulo = "_T"
palabraClaveDescripcion= "_D"
X_train_index = X_train_token.copy()
X_train_index['index descripcion'] = X_train_token['palabras descripcion'].map(lambda x : agregarYDevolverIndex(x, indexPalabrasDescripcion, palabraClaveDescripcion))
X_test_index = X_test_token.copy()
X_test_index['index descripcion'] = X_test_token['palabras descripcion'].map(lambda x : agregarYDevolverIndex(x, indexPalabrasDescripcion, palabraClaveDescripcion))

In [ ]:
X_train_index['index titulo'] = X_train_token['palabras titulo'].map(lambda x : agregarYDevolverIndex(x, indexPalabrasTitulo, palabraClaveTitulo))
X_test_index['index titulo'] = X_test_token['palabras titulo'].map(lambda x : agregarYDevolverIndex(x, indexPalabrasTitulo, palabraClaveTitulo))

In [ ]:
X_train_index.head(3)

,title,description,project,palabras descripcion,palabras titulo,index descripcion,index titulo
0,Error enabling Appcelerator services during ap...,"When creating the default app, I encountered t...",project8,"[(create, 1), (default, 1), (app, 1), (encount...","[(error, 1), (enable, 1), (appcelerator, 1), (...","[0_D, 1_D, 2_D, 3_D, 4_D]","[0_T, 1_T, 2_T, 3_T, 4_T]"
1,Create a maintenance branch,"As a developer, I'd like to have a maintenance...",project6,"[(commit, 2), (developer, 1), (like, 1), (main...","[(create, 1), (maintenance, 1), (branch, 1)]","[5_D, 6_D, 7_D, 8_D, 9_D]","[5_T, 6_T, 7_T]"
2,Service Activity Monitoring Backend integrated...,SAM API used by SAM GUI,project1,"[(sam, 2), (api, 1), (gui, 1)]","[(service, 1), (activity, 1), (monitoring, 1),...","[10_D, 11_D, 12_D]","[3_T, 8_T, 9_T, 10_T, 11_T]"


In [ ]:
print("Hay "+str(len(indexPalabrasDescripcion))+" palabras diferentes en las descripciones"),
print("Hay "+str(len(indexPalabrasTitulo))+" palabras diferentes en los titulos")

Hay 10393 palabras diferentes en las descripciones
Hay 4938 palabras diferentes en los titulos


In [ ]:
from collections import Counter

def contar(index, palabras):
    counterPalabras = Counter(palabras)
    counterIndex = Counter({key: 0 for key in index.values()})
    counterIndex.update(counterPalabras)
    return counterIndex

In [ ]:
x_train_indices = X_train_index.copy()
x_train_indices['index description count'] = x_train_indices['index descripcion'].map(lambda x :contar(indexPalabrasDescripcion, x));
x_train_indices =  x_train_indices.join(pd.json_normalize(x_train_indices['index description count']))

In [ ]:
x_test_indices = X_test_index.copy()
x_test_indices['index description count'] = x_test_indices['index descripcion'].map(lambda x :contar(indexPalabrasDescripcion, x));
x_test_indices =  x_test_indices.join(pd.json_normalize(x_test_indices['index description count']))

In [ ]:
import gc
gc.collect()

441

In [ ]:
x_train_indices['index title count'] = x_train_indices['index titulo'].map(lambda x :contar(indexPalabrasTitulo, x));
x_train_indices =  x_train_indices.join(pd.json_normalize(x_train_indices['index title count']))
x_train_indices = x_train_indices.fillna(0)

In [ ]:
x_test_indices['index title count'] = x_test_indices['index titulo'].map(lambda x :contar(indexPalabrasTitulo, x));
x_test_indices =  x_test_indices.join(pd.json_normalize(x_test_indices['index title count']))
x_test_indices = x_test_indices.fillna(0)

In [ ]:
x_train_indices.head(2)

,title,description,project,palabras descripcion,palabras titulo,index descripcion,index titulo,index description count,0_D,1_D,...,4928_T,4929_T,4930_T,4931_T,4932_T,4933_T,4934_T,4935_T,4936_T,4937_T
0,Error enabling Appcelerator services during ap...,"When creating the default app, I encountered t...",project8,"[(create, 1), (default, 1), (app, 1), (encount...","[(error, 1), (enable, 1), (appcelerator, 1), (...","[0_D, 1_D, 2_D, 3_D, 4_D]","[0_T, 1_T, 2_T, 3_T, 4_T]","{'0_D': 1, '1_D': 1, '2_D': 1, '3_D': 1, '4_D'...",1,1,...,0,0,0,0,0,0,0,0,0,0
1,Create a maintenance branch,"As a developer, I'd like to have a maintenance...",project6,"[(commit, 2), (developer, 1), (like, 1), (main...","[(create, 1), (maintenance, 1), (branch, 1)]","[5_D, 6_D, 7_D, 8_D, 9_D]","[5_T, 6_T, 7_T]","{'0_D': 0, '1_D': 0, '2_D': 0, '3_D': 0, '4_D'...",0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
x_test_indices.head(3)

,title,description,project,palabras descripcion,palabras titulo,index descripcion,index titulo,index description count,0_D,1_D,...,4928_T,4929_T,4930_T,4931_T,4932_T,4933_T,4934_T,4935_T,4936_T,4937_T
0,Add Run > Tizen Emulator menu action in App an...,The action will create the launch shortcut for...,project8,"[(launch, 2), (tizen, 2), (action, 1), (create...","[(add, 1), (run, 1), (tizen, 1), (emulator, 1)...","[75_D, 564_D, 2667_D, 0_D, 2496_D]","[32_T, 104_T, 129_T, 684_T, 182_T]","{'0_D': 1, '1_D': 0, '2_D': 0, '3_D': 0, '4_D'...",1,0,...,0,0,0,0,0,0,0,0,0,0
1,Chrome & IE mis-behavior,"On Wed, Aug 4, 2010 at 12:21 PM, Bryan Beecher...",project2,"[(bryan, 4), (d, 2), (aug, 2), (pm, 2), (beech...","[(chrome, 1), (ie, 1), (mis, 1), (behavior, 1)]","[9214_D, 2372_D, 9215_D, 3976_D, 9216_D]","[2424_T, 2788_T, 1863_T, 373_T]","{'0_D': 0, '1_D': 0, '2_D': 0, '3_D': 0, '4_D'...",0,0,...,0,0,0,0,0,0,0,0,0,0
2,Problems with Publishing routes (on release re...,I have a problem with publishing routes in Nex...,project1,"[(route, 3), (try, 2), (problem, 1), (publishi...","[(problem, 1), (publishing, 1), (route, 1), (r...","[1900_D, 2192_D, 1551_D, 5118_D, 9217_D]","[623_T, 2756_T, 445_T, 409_T, 801_T]","{'0_D': 0, '1_D': 0, '2_D': 0, '3_D': 0, '4_D'...",0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
x_train_reducido = x_train_indices.drop(["title",	"description", "palabras descripcion","palabras titulo","index descripcion","index titulo","index description count", "index title count"], axis= 1)
x_test_reducido= x_test_indices.drop(["title",	"description","palabras descripcion","palabras titulo","index descripcion","index titulo","index description count", "index title count"], axis= 1)

In [ ]:
x_train_reducido.to_csv("train_reducido.csv", index=False)
x_test_reducido.to_csv("test_reducido.csv", index=False)

In [ ]:
import os
os.kill(os.getpid(),9)

In [ ]:
import numpy as np
import pandas as pd
X_train = pd.read_csv("train_reducido.csv")
X_test= pd.read_csv("test_reducido.csv")

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
X_train['project'] = label_encoder.fit_transform(X_train['project'])

In [ ]:
X_train

,project,0_D,1_D,2_D,3_D,4_D,5_D,6_D,7_D,8_D,...,4928_T,4929_T,4930_T,4931_T,4932_T,4933_T,4934_T,4935_T,4936_T,4937_T
0,7,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6919,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6920,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6921,5,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6922,5,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_test['project'] = label_encoder.fit_transform(X_test['project'])

# Entrenamiento del Modelo y Predicciones

In [ ]:
Y_train = pd.read_csv("./train.csv")['storypoint']

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import  StratifiedKFold, KFold, RandomizedSearchCV

parametros = {'criterion':['squared_error'],
              'min_samples_split': list(range(2,15)),
              'min_samples_leaf': list(range(2,15)),
              'ccp_alpha':np.linspace(0,0.05,20),
              'max_depth':list(range(1,10)),
              'n_estimators': list(range(10,100,10)),}
numeroFolds=10
kFold = StratifiedKFold(n_splits=numeroFolds)
forestBase =  RandomForestRegressor(random_state=13, oob_score =True)
scorer = 'neg_mean_squared_error'
randomCv = RandomizedSearchCV(estimator=forestBase,
                              param_distributions = parametros,
                              scoring=scorer,
                              cv=kFold,
                              n_jobs =-1,
                              n_iter=10
                              )
model = randomCv.fit(X_train, Y_train)

NameError: name 'np' is not defined

In [ ]:
randomCv.best_params_

In [ ]:
randomCv.best_score_

In [ ]:
modelRMSE = randomCv.best_estimator_
modelRMSE

## Guardamos modelo con pickle

In [ ]:
import pickle

model_pkl_file = "random-forest.pkl"

with open(model_pkl_file, 'wb') as file:
    pickle.dump(modelRMSE, file)

NameError: name 'modelRMSE' is not defined

In [ ]:
y_test_pred = modelRMSE.predict(X_test)
print(y_test_pred)

In [ ]:
y_test_pred_rounded = np.round(y_test_pred).astype(int)

In [ ]:
test_set = pd.read_csv("./test.csv")
results = pd.DataFrame({
    'id': test_set['id'],
    'storypoint': y_test_pred_rounded
})

results.head(15)

results.to_csv('results.csv', index=False, header=True)

In [ ]:
x_train_pred = modelRMSE.predict(X_train)
print(x_train_pred)
x_train_pred_rounded = np.round(x_train_pred).astype(int)

## Metricas

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(Y_train, x_train_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_train, x_train_pred)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

metricas = {
    'metrica' : ["MSE", "RMSE", "MAE"],
    'valor' :  [mse, rmse, mae]
}
df_metrics = pd.DataFrame(metricas)
print(df_metrics)